In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/merged.csv')

In [3]:
### Season to test results

N = 2021

In [4]:
params_to_drop = ['season', 'round', 'driver', 'constructor', 'circuit_id', 'podium', 'driver_points_from']

In [5]:
data.columns

Index(['season', 'round', 'podium', 'driver', 'constructor', 'circuit_id',
       'driver_points_from', 'qualifying_pos', 'starting_grid', 'stage_q3',
       'driver_points_per', 'constructor_points_per',
       'constructor_points_before', 'driver_points_before',
       'driver_wins_before', 'constructor_wins_before',
       'constructor_standings_before', 'driver_standings_before',
       'driver_last_3', 'constructor_last_3'],
      dtype='object')

In [6]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]
X_train = train.drop(params_to_drop, axis=1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [7]:
X_train

,qualifying_pos,starting_grid,stage_q3,driver_points_per,constructor_points_per,constructor_points_before,driver_points_before,driver_wins_before,constructor_wins_before,constructor_standings_before,driver_standings_before,driver_last_3,constructor_last_3
0,-1.294774,-1.294941,1.034890,-0.809546,-0.839836,-0.692428,-0.679459,-0.314850,-0.362466,-1.413821,-1.461744,-0.760054,-0.752773
1,-1.123528,-1.123101,1.034890,-0.809546,-0.839836,-0.692428,-0.679459,-0.314850,-0.362466,-1.413821,-1.461744,-0.760054,-0.752773
2,0.075193,-0.092062,-0.966286,-0.809546,-0.839836,-0.692428,-0.679459,-0.314850,-0.362466,-1.413821,-1.461744,-0.760054,-0.752773
3,-0.952282,-0.951261,1.034890,-0.809546,-0.839836,-0.692428,-0.679459,-0.314850,-0.362466,-1.413821,-1.461744,-0.760054,-0.752773
4,-0.096053,0.767138,1.034890,-0.809546,-0.839836,-0.692428,-0.679459,-0.314850,-0.362466,-1.413821,-1.461744,-0.760054,-0.752773
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2737,0.588931,0.595298,-0.966286,-0.762335,-0.792096,-0.632629,-0.620844,-0.314850,-0.362466,0.877371,1.079251,-0.760054,-0.752773
2738,1.616406,1.282658,-0.966286,-0.809546,-0.839836,-0.692428,-0.679459,-0.314850,-0.362466,1.531997,1.714499,-0.760054,-0.752773
2739,1.102669,1.626338,-0.966286,-0.797743,-0.821553,-0.670003,-0.664805,-0.314850,-0.362466,1.204684,1.555687,-0.760054,-0.752773
2740,1.445161,1.110818,-0.966286,-0.809546,-0.687475,-0.670003,-0.679459,-0.314850,-0.362466,1.204684,2.032124,-0.760054,-0.670118


In [8]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.podium
        
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if x == prediction_df.proba_1.max() else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == N]['round'].unique().max()
    return model_score

In [9]:
model_scores ={'model':[],
                  'hidden_layer_sizes': [],
                  'activation': [],
                  'solver': [],
                  'alpha': [],
                  'score': []
                  }

In [10]:
# Neural Network Classifier training

params={'hidden_layer_sizes': [(80,20,40,5), (75,25,50,10)], 
        'activation': ['identity', 'logistic', 'tanh', 'relu'], 
        'solver': ['lbfgs', 'sgd', 'adam'], 
        'alpha': np.logspace(-4,2,20)} 


for hidden_layer_sizes in params.hidden_layer_sizes:
    for activation in params.activation:
        for solver in params.solver:
            for alpha in params.alpha:
                model_params = (hidden_layer_sizes, activation, solver, alpha)
                model = MLPClassifier(
                    hidden_layer_sizes=hidden_layer_sizes,
                    activation=activation, 
                    solver=solver, 
                    alpha=alpha, 
                    random_state=1)

                model.fit(X_train, y_train)

                model_score = score_classification(model)

                model_scores['model'].append('neural_network_classifier')
                model_scores['hidden_layer_sizes'].append(hidden_layer_sizes)
                model_scores['activation'].append(activation)
                model_scores['solver'].append(solver)
                model_scores['alpha'].append(alpha)
                model_scores['score'].append(model_score)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

In [11]:
comparison_df = pd.DataFrame(model_scores).sort_values('score', ascending=False)

comparison_df.head()

,model,hidden_layer_sizes,activation,solver,alpha,score
203,neural_network_classifier,"(80, 20, 40, 5)",relu,sgd,0.000886,0.636364
208,neural_network_classifier,"(80, 20, 40, 5)",relu,sgd,0.033598,0.636364
274,neural_network_classifier,"(75, 25, 50, 10)",identity,sgd,2.636651,0.636364
293,neural_network_classifier,"(75, 25, 50, 10)",identity,adam,1.274275,0.636364
34,neural_network_classifier,"(80, 20, 40, 5)",identity,sgd,2.636651,0.636364


model	hidden_layer_sizes	activation	solver	alpha	score
421	neural_network_classifier	(75, 25, 50, 10)	relu	lbfgs	0.000207	0.5
160	neural_network_classifier	(80, 20, 40, 5)	tanh	adam	0.000100	0.5
162	neural_network_classifier	(80, 20, 40, 5)	tanh	adam	0.000428	0.5
163	neural_network_classifier	(80, 20, 40, 5)	tanh	adam	0.000886	0.5
164	neural_network_classifier	(80, 20, 40, 5)	tanh	adam	0.001833	0.5

In [12]:
chosen_inputs = comparison_df.query('score >= 0.5')

chosen_inputs.to_csv(path+'parameters/neutralnetworkclassifier.csv')